Description: Lorem ipsum dolor sit amet consectetur adipiscing elit. Quisque faucibus ex sapien vitae pellentesque sem placerat. In id cursus mi pretium tellus duis convallis. Tempus leo eu aenean sed diam urna tempor. Pulvinar vivamus fringilla lacus nec metus bibendum egestas. Iaculis massa nisl malesuada lacinia integer nunc posuere. Ut hendrerit semper vel class aptent taciti sociosqu. Ad litora torquent per conubia nostra inceptos himenaeos.

Created by: User name, date

In [4]:
#############
# VARIABLES #
#############

COMMONS_CATEGORY = "Sprengel_Museum_Interior_Architecture"

COMMONS_API = "https://commons.wikimedia.org/w/api.php"
WIKIDATA_API = "https://www.wikidata.org/w/api.php"

In [5]:
import requests, folium, time
from folium import plugins
from folium.plugins import BeautifyIcon, MarkerCluster

################
# GETTING DATA #
################

def get_files(category, limit=50):
    """
    Fetches up to `limit` files from a Wikimedia Commons category.
    Returns a list of file names (titles).
    """
    r = requests.get(
        COMMONS_API, 
        params={
            "action": "query",
            "format": "json",
            "list": "categorymembers",
            "cmtitle": f"Category:{category}",
            "cmlimit": limit,
            "cmtype": "file",
            "nocache": str(time.time()) # avoid caching, force fresh results
        }
    )
    return r.json()["query"]["categorymembers"]

def get_coords(title):
    """
    Retrieves latitude/longitude for a Commons file.
    Returns (lat, lon) as floats, or (None, None) if not found.
    """
    r = requests.get(
        COMMONS_API,
        params={
            "action": "query",
            "titles": title,
            "prop": "coordinates",
            "colimit": "1",
            "format": "json",
            "nocache": str(time.time())
        }
    ).json()
    for p in r["query"]["pages"].values():
        if "coordinates" in p:
            return p["coordinates"][0]["lat"], p["coordinates"][0]["lon"]
    return None, None

def get_description(title):
    """
    Fetches the description for a Commons file from structured metadata.
    Returns the description string, or fallback if missing.
    """
    r = requests.get(
        COMMONS_API,
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "imageinfo",
            "iiprop": "extmetadata",
        }
    ).json()
    for p in r["query"]["pages"].values():
        meta = p.get("imageinfo", [{}])[0].get("extmetadata", {})
        return meta.get("ImageDescription", {}).get("value", "No description available.")
    return "No description found."

def get_url(title):
    """
    Retrieves the direct image URL for a Commons file.
    Returns a string URL.
    """
    r = requests.get(
        COMMONS_API,
        params={
            "action": "query",
            "titles": title,
            "prop": "imageinfo",
            "iiprop": "url",
            "format": "json"
        }
    ).json()
    for p in r["query"]["pages"].values():
        return p["imageinfo"][0]["url"]


def get_depicts_qid(title):
    """
    Retrieves the 'depicts' (P180) Wikidata Q-ID for a Commons file.
    Returns one Q-ID string (or empty if none found).
    """
    # 1. FIND PAGE_ID BY TITLE
    r1 = requests.get(
        COMMONS_API,
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "info"
        }
    )
    r1.raise_for_status()
    pages = r1.json().get("query", {}).get("pages", {})
    media_id = None
    for page_id, page in pages.items():
        if page.get("missing"):
            continue
        # CONSTRUCT MEDIA_ID
        media_id = f"M{page_id}"
        break
    if not media_id:
        return []

    # 2. FIND STRUCTURED DATA BY MEDIA_ID
    r2 = requests.get(
        COMMONS_API,
        params={
            "action": "wbgetentities",
            "format": "json",
            "ids": media_id,
            "props": "claims"
        }
    )
    r2.raise_for_status()
    entities = r2.json().get("entities", {})
    ent = entities.get(media_id, {})

    # 3. EXTRACT P180 (DEPICTS) Q-ID FROM STRUCTURED DATA
    # (CONNECTION TO A WIKIDATA ENTRY)
    statements = ent.get("statements", {})
    qid = ''
    for claim in statements.get("P180", []):
        dv = claim.get("mainsnak", {}).get("datavalue", {})
        value = dv.get("value", {})
        qid = value.get("id")
    return qid

########################
# WIKIDATA / WIKIPEDIA #
########################

def get_wikipedia_url_from_qid(qid, lang="en"):
    """
    Given a Wikidata Q-ID (e.g. "Q510144") and language (default: en),
    returns the Wikipedia URL for that entity in the specified language.
    Returns None if entity is not present.
    """
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "sitelinks",
        "sitefilter": f"{lang}wiki",  
        "format": "json"
    }
    r = requests.get(WIKIDATA_API, params=params, timeout=10)
    r.raise_for_status()
    # FIND WIKIPEDIA LINK ON WIKIDATA
    data = r.json().get("entities", {}).get(qid, {})
    sitelinks = data.get("sitelinks", {})
    entry = sitelinks.get(f"{lang}wiki")
    if not entry:
        return None
    # CONSTRUCT URL IF THERE'S NO LINK LISTED
    url = entry.get("url")
    if url:
        return url
    title = entry.get("title", "").replace(" ", "_")
    return f"https://{lang}.wikipedia.org/wiki/{title}" if title else None

#######################
# PUTTING IT TOGETHER #
#######################

def get_geotagged_files(category, target=9, batch=50):
    """
    Finds up to `target` files in a category that have geo-coordinates.
    For each: collects file title, lat/lon, image URL, description, and depicts Q-ID.
    Returns a list of tuples: (title, lat, lon, img_url, description, qid)
    """
    files = get_files(category, limit=batch)
    locations = []
    for f in files:
        lat, lon = get_coords(f["title"])
        if lat is not None:
            locations.append(
                (
                    f["title"],
                    lat,
                    lon,
                    get_url(f["title"]),
                    get_description(f["title"]),
                    get_depicts_qid(f["title"])
                )
            )
            if len(locations) == target:
                break
        time.sleep(0.2)  
    return locations

locations = get_geotagged_files(COMMONS_CATEGORY, target=9)

##############
# MAP OUTPUT #
##############

if not locations:
    print("No structured geocoordinates found.")
else:
    # INITIALISE MAP (CENTER @ FIRST PAIR OF COORDINATES)
    m = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=16, max_zoom=25)
    # CLUSTER MARKERS TOGETHER
    marker_cluster = MarkerCluster().add_to(m)

# ITERATE THROUGH INFO LIST
for idx, (name, lat, lon, img, desc, qid) in enumerate(locations, start=1):

    # CLEAN UP FILE NAME
    label = name.replace('File:', '').rsplit('.', 1)[0]

    # HTML POPUP WITH IMAGE AND TITLE
    popup_html = f"""
    <h4 style="text-align: center; font-weight:bold">{label}</h4><br>
    <img src="{img}" style="
        width: 200px;
        height: 200px;
        object-fit: cover;
        border-radius: 100px;
        display: block;
        margin: auto;
    ">
    """

    # PLACE MARKER
    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_html, max_width=220),
        icon=BeautifyIcon(
            icon_shape='marker', number=idx, background_color='lightblue'
        ),
        tooltip=label
    ).add_to(marker_cluster)

# ADD FULLSCREEN TO MAP
folium.plugins.Fullscreen(
    position="topright",
    title="Vollbild",
    title_cancel="Schließen",
    force_separate_button=True,
).add_to(m)

m


In [6]:
##############
# INFO TABLE #
##############

# TABLE HEADER
print("""
```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:5%">Nr.</th>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Beschreibung</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:5%"> </th> 
      </tr>
      </thead>
      <tbody>
```
      """)

# ITERATE THROUGH INFO LIST AGAIN
for idx, (name, lat, lon, img, desc, qid) in enumerate(locations, start=1):
  clean = name.replace('File:', '').rsplit('.', 1)[0]

  # GET WIKI LINK
  wiki = get_wikipedia_url_from_qid(qid, lang="en")

  # NUMBERING
  print(f"""
```{{=html}}
<tr>
<td><b>{idx}</b></td>
```
  """)

  # TITLE & DESCRIPTION 
  print(f"""
```{{=html}}
<td><span style='display:none'>{name}</span><a href='https://commons.wikimedia.org/wiki/{name}'>{clean}</a></td>
<td>{desc}</td> 
```
""")

  # COORDINATES 
  print(f"""
```{{=html}}
  <td>{lat},<br> {lon}</td>
```
""")
  
  # LINK TO RESONATOR IF Q-ID EXISTS
  if qid != '':
    print(f"""
```{{=html}}
<td><a href="https://reasonator.toolforge.org/?q={qid}&lang=mul"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Reasonator_logo_proposal.png/24px-Reasonator_logo_proposal.png"></a>
```
""")
  # LINK TO WIKIPEDIA PAGE IF ONE EXISTS
    if wiki:
      print(f"""
```{{=html}}
<br><a href="{wiki}"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/Wikipedia%27s_W.svg" height="24px"></a></td>
```    
""")
    else: 
      print("""
```{=html}
</td>
```
      """)
  else:
    print("""
```{=html}
<td></td>
```
    """)

  print("""
```{=html}
</tr>
```
""")
print("""
```{=html}
</tbody></table>
```
""")

# JAVASCRIPT FOR TABLE SORTING
print("""
<script>
function sortTable(n) {
  var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0;
  table = document.getElementById("mapTable");
  switching = true;
  // Set the sorting direction to ascending:
  dir = "asc";
  /* Make a loop that will continue until
  no switching has been done: */
  while (switching) {
    // Start by saying: no switching is done:
    switching = false;
    rows = table.rows;
    /* Loop through all table rows (except the
    first, which contains table headers): */
    for (i = 1; i < (rows.length - 1); i++) {
      // Start by saying there should be no switching:
      shouldSwitch = false;
      /* Get the two elements you want to compare,
      one from current row and one from the next: */
      x = rows[i].getElementsByTagName("TD")[n];
      y = rows[i + 1].getElementsByTagName("TD")[n];
      /* Check if the two rows should switch place,
      based on the direction, asc or desc: */
      if (dir == "asc") {
        if (x.innerHTML.toLowerCase() > y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      } else if (dir == "desc") {
        if (x.innerHTML.toLowerCase() < y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      }
    }
    if (shouldSwitch) {
      /* If a switch has been marked, make the switch
      and mark that a switch has been done: */
      rows[i].parentNode.insertBefore(rows[i + 1], rows[i]);
      switching = true;
      // Each time a switch is done, increase this count by 1:
      switchcount ++;
    } else {
      /* If no switching has been done AND the direction is "asc",
      set the direction to "desc" and run the while loop again. */
      if (switchcount == 0 && dir == "asc") {
        dir = "desc";
        switching = true;
      }
    }
  }
}
</script>
""") 


```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:5%">Nr.</th>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Beschreibung</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:5%"> </th> 
      </tr>
      </thead>
      <tbody>
```
      

```{=html}
<tr>
<td><b>1</b></td>
```
  

```{=html}
<td><span style='display:none'>File:Corridor in Sprengel Museum.jpg</span><a href='https://commons.wikimedia.org/wiki/File:Corridor in Sprengel Museum.jpg'>Corridor in Sprengel Museum</a></td>
<td>Example of brutalist architecture</td> 
```


```{=html}
  <td>52.363108,<br> 9.740197</td>
```


```{=html}
<td><a href="https://reasonator.toolforge.org/?q=Q510144&lang=mul"><img src="https